In [ ]:
---
title: "Data Cleaning"
format: html
toc: true
toc_depth: 3 
number sections: true 
code-fold: show
---

<b>*Disclaimer - this section is a work in progress</b>
<br></br>
This page shows the raw data, the code used to clean it, and the modified data. It's a journal of my data cleaning process. Please be aware that this page contains both Python and R code, thus you should avoid running the source code all at once.

# Baseballr


# ncaahoopR

let's clean the Villanova 2021-22 data with R:

here is a screen shot of the first few rows and columns of the raw data:
<br></br>
![raw data](./images/villanova2122raw.png)

In [5]:
library(tidyverse)

-- Attaching core tidyverse packages ------------------------ tidyverse 2.0.0 --
v dplyr     1.1.2     v readr     2.1.4
v forcats   1.0.0     v stringr   1.5.0
v ggplot2   3.4.2     v tibble    3.2.1
v lubridate 1.9.2     v tidyr     1.3.0
v purrr     1.0.1     
-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()
i Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [1]:
# let's load in the data
nova2122 <- read.csv('./data/raw_data/villanova2122.csv')

In [2]:
# let's check the shape of the data
dim(nova2122)

[1] 11405    39

In [3]:
# what are the column names?
colnames(nova2122)

[1] "game_id"                 "date"                   
 [3] "home"                    "away"                   
 [5] "play_id"                 "half"                   
 [7] "time_remaining_half"     "secs_remaining"         
 [9] "secs_remaining_absolute" "description"            
[11] "action_team"             "home_score"             
[13] "away_score"              "score_diff"             
[15] "play_length"             "scoring_play"           
[17] "foul"                    "win_prob"               
[19] "naive_win_prob"          "home_time_out_remaining"
[21] "away_time_out_remaining" "home_favored_by"        
[23] "total_line"              "referees"               
[25] "arena_location"          "arena"                  
[27] "capacity"                "attendance"             
[29] "shot_x"                  "shot_y"                 
[31] "shot_team"               "shot_outcome"           
[33] "shooter"                 "assist"                 
[35] "three_pt"                "free_throw"             
[37] "possession_before"       "possession_after"       
[39] "wrong_time"

In [6]:
# this data looks relatively clean, but we want only shooting data
# let's get rid of rows where there isn't a shooter
# this would be rows where the shooter is NA
# such as a turnover, steal, rebound, or block
nova2122 <- nova2122 %>%
  filter(!is.na(shooter))

# let's check the shape of the data
dim(nova2122)

[1] 5399   39

In [8]:
# we can see that we removed about 5,000 rows and are left with just a little over half the initial data

# I want to create a previous_shots column that says how many shots the shooter has made or missed in a row before the current shot they are taking
sample <- nova2122 %>% select(game_id, play_id, half, shooter, shot_outcome)

sample <- sample %>%
  mutate(
    shot_outcome_numeric = ifelse(shot_outcome == "made", 1, -1)
  )

sample <- sample %>%
  group_by(game_id, half, shooter) %>%
  arrange(play_id) %>%
  mutate(
    shot_sequence = cumsum(shot_outcome_numeric)) %>%
  ungroup()

sample3 <- sample %>%
  mutate(
    shot_sequence = ifelse(shot_outcome == "made" & shot_sequence <= 0, 1,
                  ifelse(shot_outcome == "missed" & shot_sequence >= 0, -1, shot_sequence))
  )

sample3 <- sample3 %>%
  group_by(game_id, half, shooter) %>%
  arrange(play_id) %>%
  mutate(
    previous_shots = ifelse(row_number() == 1, 0, lag(shot_sequence, default = 0))
  ) %>%
  ungroup()

write.csv(sample3, file = "./data/modified_data/nova2122.csv", row.names = FALSE)

Here is a screen shot of the modified data: 
<br></br>
![modified data](./images/nova2122clean.png)

# reddit

# newsapi

let's clean this using python:
<br></br>
here is a picture of the first few rows of the raw data:
<br></br>
![raw data](./images/newsapi.png)

In [4]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
#reading in the file
newsapi = pd.read_csv('./data/raw_data/newsapi.csv')

In [3]:
# what is the shape of this data
newsapi.shape

(100, 4)

In [8]:
# what are the column names
newsapi.columns

Index(['0', '1', '2', '3'], dtype='object')

In [14]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/williammcgloin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/williammcgloin/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/williammcgloin/nltk_data...


True

In [15]:
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Initialize the Lemmatizer and stopwords list
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    # Remove special characters and numbers
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    
    # Tokenization and lowercase
    words = text.lower().split()
    
    # Remove stopwords and apply lemmatization
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    
    return ' '.join(words)

# Apply preprocessing to the 'text' column
newsapi['cleaned_text'] = newsapi['2'].apply(preprocess_text)

In [27]:
# Initialize the CountVectorizer
count_vectorizer = CountVectorizer()

# Fit and transform the preprocessed text data
X = count_vectorizer.fit_transform(newsapi['cleaned_text'])

# printing part of the sparse matrix
print(X[:20, :20].toarray())

[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0]]


In [26]:
# Convert the sparse matrix to a Pandas DataFrame
X_df = pd.DataFrame(X.toarray())

# Display the first few rows of the DataFrame
print(X_df.head())

   0    1    2    3    4    5    6    7    8    9    ...  539  540  541  542  \
0    0    0    0    0    0    0    0    0    0    0  ...    0    0    0    0   
1    0    0    0    0    0    0    0    0    0    0  ...    0    0    0    0   
2    0    0    0    0    0    0    0    0    0    0  ...    0    0    0    0   
3    0    0    0    0    0    0    0    0    0    0  ...    0    0    0    0   
4    0    0    0    0    0    0    0    0    0    0  ...    0    0    0    0   

   543  544  545  546  547  548  
0    0    0    0    0    0    0  
1    0    0    0    0    0    0  
2    0    0    0    0    0    0  
3    0    0    0    0    0    0  
4    0    0    0    0    0    0  

[5 rows x 549 columns]


In [18]:
vocab = count_vectorizer.get_feature_names_out()

print(vocab)

['abnormality' 'abound' 'ac' 'acc' 'action' 'advice' 'aew' 'ahead' 'al'
 'alds' 'alert' 'already' 'alyssa' 'amid' 'andy' 'angelos' 'another'
 'answer' 'apple' 'argentina' 'arizona' 'arkansas' 'armondo' 'arsenal'
 'astonishing' 'austin' 'back' 'balogun' 'ban' 'baseball' 'battle' 'bear'
 'bearcat' 'beat' 'beaten' 'become' 'becoming' 'behind' 'belief'
 'bellingham' 'bengal' 'besides' 'best' 'bet' 'better' 'beyond' 'big'
 'biggs' 'bird' 'blount' 'blue' 'blunder' 'bonus' 'booing' 'booking'
 'boston' 'bottom' 'brad' 'brain' 'brave' 'breaking' 'breanna' 'brewer'
 'buccaneer' 'building' 'bukayo' 'bumper' 'call' 'candidate' 'cant' 'card'
 'cardinal' 'cargill' 'carlos' 'case' 'casino' 'catch' 'central' 'cfb'
 'champion' 'charge' 'chicago' 'christian' 'cincinnati' 'city' 'climate'
 'clinch' 'close' 'closer' 'clue' 'clutch' 'coach' 'coaching'
 'cockeysville' 'college' 'colorado' 'column' 'come' 'coming' 'commander'
 'commits' 'comparison' 'complete' 'concern' 'conference' 'contender'
 'corner' 'co

# individual player data

let's clean the aaron judge game data with python:

here is a screen shot of the first few rows of the raw data:
<br></br>
![raw data](./images/judge2.png)

In [ ]:
import pandas as pd
import numpy as np

In [80]:
#reading in the file
aaronjudge = pd.read_csv('./data/raw_data/AaronJudgeData.csv')

In [81]:
#how many rows are in this dataset?
aaronjudge.shape

(111, 37)

In [82]:
#what are the column names?
aaronjudge.columns

Index(['Date', 'Team', 'Opp', 'BO', 'Pos', 'PA', 'H', '2B', '3B', 'HR', 'R',
       'RBI', 'SB', 'CS', 'BB%', 'K%', 'ISO', 'BABIP', 'EV', 'AVG', 'OBP',
       'SLG', 'wOBA', 'wRC+', 'Date.1', 'Team.1', 'Opp.1', 'BO.1', 'Pos.1',
       'Events', 'EV.1', 'maxEV', 'LA', 'Barrels', 'Barrel%', 'HardHit',
       'HardHit%'],
      dtype='object')

In [83]:
#removing the repeated columns
columns_to_remove = ['Date.1', 'Team.1', 'Opp.1', 'BO.1', 'Pos.1']
aaronjudge.drop(columns=columns_to_remove, inplace=True)
aaronjudge.columns

Index(['Date', 'Team', 'Opp', 'BO', 'Pos', 'PA', 'H', '2B', '3B', 'HR', 'R',
       'RBI', 'SB', 'CS', 'BB%', 'K%', 'ISO', 'BABIP', 'EV', 'AVG', 'OBP',
       'SLG', 'wOBA', 'wRC+', 'Events', 'EV.1', 'maxEV', 'LA', 'Barrels',
       'Barrel%', 'HardHit', 'HardHit%'],
      dtype='object')

In [84]:
# i belive the initial row with the column names is repeated throughou the data. let's check
print((aaronjudge['Date'] == 'Date').sum())

5


In [85]:
# let's remove these rows and then check the shape again
aaronjudge.drop(aaronjudge[aaronjudge['Date'] == 'Date'].index, inplace=True)
aaronjudge.shape

(106, 32)

In [86]:
# there is also a total row which I want to remove as well. let's do that now
aaronjudge.drop(aaronjudge[aaronjudge['Date'] == 'Total'].index, inplace=True)
aaronjudge.shape

(105, 32)

In [87]:
# so far, I have removed 6 rows and 5 columns. 

# I want to create a "location" column based on the "@" in the "Opp" column
aaronjudge['location'] = aaronjudge['Opp'].apply(lambda x: 'away' if '@' in x else 'home')

# Remove the "@" symbol from the values in the "Opp" column
aaronjudge['Opp'] = aaronjudge['Opp'].str.replace('@', '')

# check value counts of the new "location" column
print(aaronjudge['location'].value_counts()) #this seems accurate

home    53
away    52
Name: location, dtype: int64


In [88]:
print(aaronjudge['PA'].dtype)
print(aaronjudge['BB%'].dtype)

object
object


In [89]:
# I want to create two new columns. The number of at bats per each game and the number of hard hits in each game. 
# for this project, we are going to calculate at-bats as should be the number of plate appearances minus walks (sacrifices and HBP are not included in this dataset)

#first i have to remove the '%' symbol and convert 'BB%' to a float

aaronjudge['BB%'] = aaronjudge['BB%'].astype(str)
aaronjudge['BB%'] = aaronjudge['BB%'].str.rstrip('%').astype(float) / 100.0

# Round the 'BB%' column to three decimal places
aaronjudge['BB%'] = aaronjudge['BB%'].round(3)

#print(aaronjudge['BB%'].mean())

#convert 'PA' to a float
aaronjudge['PA'] = aaronjudge['PA'].astype(float)

# now I can create the new at_bats column
aaronjudge['at_bats'] = aaronjudge['PA'] * (1 - aaronjudge['BB%'])

#now lets see the average number of at bats vs the average number of plate appearances
print(aaronjudge['at_bats'].mean())
print(aaronjudge['PA'].mean())

3.4857333333333336
4.314285714285714


In [90]:
# now I want to create a new column for hard hits per game
# we can do this by multiplying the hard hit percentage by the events column (these columns were part of a different table that was merged with the original table)

print(aaronjudge['HardHit%'].dtype)
print(aaronjudge['Events'].dtype)

object
object


In [91]:
# this code is very similar to what we just did

#first i have to remove the '%' symbol and convert 'HardHit%' to a float

aaronjudge['HardHit%'] = aaronjudge['HardHit%'].astype(str)
aaronjudge['HardHit%'] = aaronjudge['HardHit%'].str.rstrip('%').astype(float) / 100.0

# Round the 'HardHit%' column to three decimal places
aaronjudge['HardHit%'] = aaronjudge['HardHit%'].round(3)

#print(aaronjudge['HardHit%'].mean())

#convert 'Events' to a float
aaronjudge['Events'] = aaronjudge['Events'].astype(float)

# now I can create the new hard_hits column
aaronjudge['hard_hits'] = (aaronjudge['Events'] * aaronjudge['HardHit%']).round(0)

#now lets see the average number of hard_hits per game
print(aaronjudge['hard_hits'].mean())

1.52


In [92]:
# finally, let's create a correct hardhit% column that is based on the number of at-bats, not the number of times a player puts the ball in play
aaronjudge['correct_hardhit%'] = (aaronjudge['hard_hits'] / aaronjudge['at_bats']).round(2)

# now let's see the average correct hardhit% for Aaron Judge
print(aaronjudge['correct_hardhit%'].mean())

0.42829999999999996


In [95]:
# sometimes in certain stadiums or based on the weather, the HardHit% data is missing
# this causes the value of the new correct_hardhit% column to be NaN, so let's remove those few rows
aaronjudge.dropna(subset=['correct_hardhit%'], inplace=True)

#let's check the shape again
aaronjudge.shape #loss of 5 rows

(100, 36)

In [96]:
# now we can save this to a csv file
aaronjudge.to_csv('./data/modified_data/aaronjudge.csv', index=False)

here is a screenshot of the first couple rows of the modified csv file:
<br></br>
![Clean Data](./images/aaron_judge_clean.png)

# Extra Joke

What did the broom say to the vacuum?

“I’m so tired of people pushing us around.”
<br></br>
![Figure 1](./images/broom-water-buckets.png)